In [1]:
import os
import random

import numpy as np
#import pandas as pd
import matplotlib.pyplot as plt

#from tqdm import tqdm_notebook, tnrange
#from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf
print(tf.__version__)
print(tf.executing_eagerly())

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

/Users/vetlebh/miniconda3/envs/model/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Sequence
/Users/vetlebh/miniconda3/envs/model/lib/python3.7/site-packages/sklearn/model_selection/_split.py:18: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/Users/vetlebh/miniconda3/envs/model/lib/python3.7/site-packages/sklearn/model_selection/_search.py:16: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping, namedtuple, defaultdict, Sequence


2.2.0
True


In [2]:
from spline import Spline
from network import get_model

### Data Generator

In [3]:
im_height = 256
im_width = 256

checkpoint_path = 'weights_1.h5'
model_path = 'model_1.h5'

epochs = 1
batch_size = 1

training_path = 'training.nosync'
img_name = 'image.npy'
gt_name = 'gt.npy'

test_size = 0.1
val_size = 0.1
random_state = 69

max_slices = 15





walk = next(os.walk(training_path))[1]

X = np.zeros((len(walk)*max_slices, im_height, im_width, 1))
y = np.zeros((len(walk)*max_slices, im_height, im_width, 1))

img_nr = 0
sum_slices = 0
patients_not_found = 0
for ids in walk[:2]:
    
    try:
        img = np.load(os.path.join(training_path, ids, img_name))
        gt = np.load(os.path.join(training_path, ids, gt_name))
        slices = img.shape[2]
        
        for slice_nr in range(slices):
            
            img_slice, gt_slice = img[:, :, slice_nr], gt[:, :, slice_nr]
            img_resized = resize(img_slice, (im_height, im_width, 1), mode = 'edge', preserve_range = True, anti_aliasing=True)
            gt_resized = resize(gt_slice, (im_height, im_width, 1), mode = 'edge', preserve_range = True, anti_aliasing=True)
            
            # We are only interested in the classes 'heart' and 'background' for this experiment
            gt_resized = (gt_resized > 0.5).astype(np.uint8)
        
            X[sum_slices, :, :, :] = img_resized/255.0
            y[sum_slices, :, :, :] = gt_resized
            
            sum_slices +=1
     
    except:
        print(f'{ids} not found')
        patients_not_found += 1
        continue
    
    if(img_nr%10 == 0):
        print(f'{img_nr} images and {sum_slices} slices loaded to array')
    img_nr += 1
print(f'Image load complete. {img_nr} images and {sum_slices} slices loaded successfully. ')

0 images and 9 slices loaded to array
Image load complete. 2 images and 19 slices loaded successfully. 


In [4]:
X, y = X[:sum_slices, :, :, :], y[:sum_slices, :, :, :]
print(X.shape, y.shape)
print(np.unique(y))

# Split train data and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

# Split train data into train and valid
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=val_size, random_state=random_state)

print(f'Training size: {X_train.shape[0]}, Validation size: {X_valid.shape[0]}, Test size: {X_test.shape[0]}')

(19, 256, 256, 1) (19, 256, 256, 1)
[0. 1.]
Training size: 15, Validation size: 2, Test size: 2


### Model

In [5]:
model = get_model()

# Evaluate the model pre training
loss, acc = model.evaluate(X_test,  y_test, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

In [6]:
y_test.shape
y_pred = model.predict(X_test)

### Train

In [7]:
callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1),
    ModelCheckpoint(checkpoint_path, verbose=1, save_best_only=True, save_weights_only=True)
]

In [8]:
results = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, callbacks=callbacks, validation_data=(X_valid, y_valid))

AttributeError: in user code:

    /Users/vetlebh/miniconda3/envs/model/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /Users/vetlebh/miniconda3/envs/model/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/vetlebh/miniconda3/envs/model/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/vetlebh/miniconda3/envs/model/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/vetlebh/miniconda3/envs/model/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:533 train_step  **
        y, y_pred, sample_weight, regularization_losses=self.lo

### Evaluate

In [ ]:
# Loads the weights
model.load_weights(checkpoint_path)

In [ ]:
# Evaluate the model post training
loss, acc = model.evaluate(X_test,  y_test, verbose=2)
print("Trained model, accuracy: {:5.2f}%".format(100*acc))

In [ ]:
model.save(model_path)

In [ ]:
from network import pixelizedIoU

In [ ]:
loss = pixelizedIoU()
print(y_test[1])

In [ ]:
loss(y_test[:,:,:,:], y_pred)

In [ ]:
y_pred

In [ ]:
spline = Spline(y_pred.transpose())

In [ ]:
spline.drawCurve()